## Section 1: Operator Overview 
* Using the most recent month

In [1]:
import _report_utils
import _section1_utils as section1
import altair as alt
import calitp_data_analysis.magics
import geopandas as gpd
import great_tables as gt
import pandas as pd
from calitp_data_analysis import calitp_color_palette as cp
from calitp_data_analysis.sql import to_snakecase
from great_tables import md
from IPython.display import HTML, Markdown, display
from segment_speed_utils.project_vars import RT_SCHED_GCS
from shared_utils import catalog_utils, rt_dates, rt_utils

In [2]:
alt.renderers.enable("html")
alt.data_transformers.enable("default", max_rows=None)

DataTransformerRegistry.enable('default')

In [3]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [4]:
name = "Bay Area 511 AC Transit Schedule"
organization_name = "Alameda-Contra Costa Transit District"

In [5]:
selected_date = pd.to_datetime(rt_dates.DATES["mar2024"])
year = selected_date.year
GTFS_DATA_DICT = catalog_utils.get_catalog("gtfs_analytics_data")

### Map

In [ ]:
ac_transit_map = section1.load_operator_map(name)

In [ ]:
ac_transit_map.shape

In [ ]:
ac_transit_map.columns

### Operator Profiles

In [ ]:
ac_transit_profile = section1.load_operator_profiles(organization_name)

In [ ]:
ac_transit_profile

In [ ]:
ac_transit_profile.name.values[0]